In [1]:
import pandas as pd

In [2]:
dataset = pd.read_csv('housing.csv')

In [3]:
print(dataset.shape)

(506, 13)


In [4]:
dataset[:5]

,crim,zn,indus,chas,nox,age,rm,dis,rad,tax,ptratio,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,5.33,36.2


In [5]:
dataset = pd.concat([dataset['medv'],
dataset.drop(['medv'], axis=1)],
axis=1)

In [6]:
dataset[:5]

,medv,crim,zn,indus,chas,nox,age,rm,dis,rad,tax,ptratio,lstat
0,24.0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,4.98
1,21.6,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,9.14
2,34.7,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,4.03
3,33.4,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,2.94
4,36.2,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,5.33


In [7]:
from sklearn.model_selection import train_test_split

In [8]:
training_dataset, validation_dataset = train_test_split(dataset, test_size=0.1)

In [9]:
training_dataset.to_csv('training_dataset.csv',index=False, header=False)
validation_dataset.to_csv('validation_dataset.csv',index=False, header=False)

In [13]:
import boto3

In [31]:
session = boto3.Session(profile_name='china')

In [32]:
session

Session(region_name='cn-northwest-1')

In [33]:
import sagemaker

In [59]:
sess = sagemaker.Session(boto_session=session)
bucket = sess.default_bucket()

In [60]:
bucket

'sagemaker-cn-northwest-1-017664957742'

In [65]:
prefix = 'boston-housing'
training_data_path = sess.upload_data(path='training_dataset.csv',key_prefix=prefix + '/input/training')
validation_data_path = sess.upload_data(path='validation_dataset.csv',key_prefix=prefix + '/input/validation')
print(training_data_path)
print(validation_data_path)

s3://sagemaker-cn-northwest-1-017664957742/boston-housing/input/training/training_dataset.csv
s3://sagemaker-cn-northwest-1-017664957742/boston-housing/input/validation/validation_dataset.csv


In [22]:
region = session.region_name
container = sagemaker.image_uris.retrieve('linear-learner', region)

In [26]:
region
container

'387376663083.dkr.ecr.cn-northwest-1.amazonaws.com.cn/linear-learner:1'

In [42]:
iam = session.client('iam')

In [43]:
role = iam.get_role(RoleName='AmazonSageMaker-ExecutionRole-20201207T092541')['Role']['Arn']

In [58]:
sess

In [73]:
ll_estimator = sagemaker.estimator.Estimator(
container,
role=role,
instance_count=1,
instance_type='ml.m5.large',
output_path='s3://{}/{}/output'.format(bucket,prefix),
sagemaker_session=sess,
disable_profiler=True)

In [74]:
ll_estimator.set_hyperparameters(
predictor_type='regressor',
mini_batch_size=32)

In [75]:
ll_estimator.fit()

2020-12-19 03:10:14 Starting - Starting the training job...
2020-12-19 03:10:19 Starting - Launching requested ML instances...
2020-12-19 03:11:21 Starting - Preparing the instances for training......
2020-12-19 03:12:23 Downloading - Downloading input data...
2020-12-19 03:13:17 Training - Downloading the training image..Docker entrypoint called with argument(s): train
Running default environment configuration script
[12/19/2020 03:13:34 INFO 140262562305856] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/resources/default-input.json: {u'loss_insensitivity': u'0.01', u'epochs': u'15', u'feature_dim': u'auto', u'init_bias': u'0.0', u'lr_scheduler_factor': u'auto', u'num_calibration_samples': u'10000000', u'accuracy_top_k': u'3', u'_num_kv_servers': u'auto', u'use_bias': u'true', u'num_point_for_scaler': u'10000', u'_log_level': u'info', u'quantile': u'0.5', u'bias_lr_mult': u'auto', u'lr_scheduler_step': u'auto', u'init_method': u'uniform', u'init_

UnexpectedStatusException: Error for Training job linear-learner-2020-12-19-03-10-12-618: Failed. Reason: ClientError: Unable to initialize the algorithm. Failed to validate input data configuration. (caused by ValidationError)

Caused by: {} is not valid under any of the given schemas

Failed validating u'anyOf' in schema:
    {u'$schema': u'http://json-schema.org/draft-04/schema#',
     u'additionalProperties': False,
     u'anyOf': [{u'required': [u'train']}, {u'required': [u'state']}],
     u'definitions': {u'data_channel': {u'properties': {u'ContentType': {u'type': u'string'},
                                                        u'RecordWrapperType': {u'enum': [u'None'],
                                                                               u'type': u'string'},
                                                        u'S3DataType': {u'enum': [u'S3Prefix',
                                                                                  u'ManifestFile'],
                 

In [79]:
training_data_channel = sagemaker.TrainingInput(
s3_data=training_data_path,
content_type='text/csv')

In [80]:
validation_data_channel = sagemaker.TrainingInput(
s3_data=validation_data_path,
content_type='text/csv')

In [81]:
ll_estimator.fit({'train': training_data_channel,'validation':validation_data_channel})

 "epoch": 11}, "StartTime": 1608351493.335521}

#metrics {"Metrics": {"train_mse_objective": {"count": 1, "max": 0.3601327581065042, "sum": 0.3601327581065042, "min": 0.3601327581065042}}, "EndTime": 1608351493.335884, "Dimensions": {"model": 17, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 11}, "StartTime": 1608351493.33586}

#metrics {"Metrics": {"train_mse_objective": {"count": 1, "max": 0.35986497146742685, "sum": 0.35986497146742685, "min": 0.35986497146742685}}, "EndTime": 1608351493.336223, "Dimensions": {"model": 18, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 11}, "StartTime": 1608351493.336198}

#metrics {"Metrics": {"train_mse_objective": {"count": 1, "max": 0.36015636899641584, "sum": 0.36015636899641584, "min": 0.36015636899641584}}, "EndTime": 1608351493.336571, "Dimensions": {"model": 19, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 11}, "StartTime": 1608351

In [85]:
from time import strftime, gmtime, localtime
timestamp = strftime('%y-%m-%d-%H-%M-%S', localtime())
endpoint_name = 'linear-learner-demo-'+timestamp
print(endpoint_name)

linear-learner-demo-20-12-19-21-08-50


In [91]:
ll_predictor = ll_estimator.deploy(
endpoint_name=endpoint_name,
initial_instance_count=1,
instance_type='ml.t2.medium')

-----------------!

In [93]:
#ll_predictor.content_type = 'text/csv'
ll_predictor.serializer = sagemaker.serializers.CSVSerializer()
ll_predictor.deserializer = sagemaker.deserializers.CSVDeserializer()

In [94]:
test_sample = '0.00632,18.00,2.310,0,0.5380,6.5750,65.20,4.0900,1,296.0,15.30,4.98'

In [95]:
response = ll_predictor.predict(test_sample)

In [96]:
print(response)

[['29.7144241333']]


In [ ]:
ll_predictor.delete_endpoint()